In [1]:
import time
import pandas as pd
import numpy as np

In [2]:
import requests
import os
import csv

In [3]:
df= pd.read_table('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/COCONUT_herb_compound_ID(remove_duplicate)_tab.txt', index_col = False)
df

,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
0,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",31598,5-(3'-Methoxycarbonylbutyroyl)aminomethyl-tran...,null,null,null,null
1,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39863,Azedarachin A,null,null,null,null
2,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39864,12-O-Acetylazedarachin A,null,null,null,null
3,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39865,Azecin 3,null,null,null,null
4,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39866,Azecin 4,null,182565-81-7,null,null
5,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39879,"(1S,2R,8R,10R,11S,15R)-21-Acetoxy-6-(3-furyl)-...",null,null,null,null
6,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39880,"1-(3-Furyl)-5-(hydroxymethyl)-8a-methyl-1,7,8,...",null,220210-98-0,null,null
7,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39881,Melianolide,null,174416-97-8,null,null
8,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39882,"Methyl [(1S,5S,7S,8S,16S)-5,11-diacetoxy-16-{[...",null,null,null,null
9,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",39883,Trichilin H,null,null,null,null


In [11]:
df = df.fillna('null')

In [12]:
# get pubchem_id of top_herb
pubchem_id = df['pubchemID'].tolist()

In [17]:
# using UniChem REST API, convert pubchem ID to KEGG ID
# 1. requests HTML

url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'

# list of r object(request result)
rs=[]
cnt = 0
print time.strftime('%a %H:%M:%S')
for i in pubchem_id:
    if i != 'null':
        url = url_base + i + '/22/6'
        #scrap the HTML at the url
        try : 
            r = requests.get(url)
            rs.append(r)
            r.connection.close()
        except requests.ConnectionError:
            print url
            break
    else:
        rs.append('null')
    cnt = cnt + 1
    if cnt % 100 == 0 :
        print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

Tue 19:50:49
100 : Tue 19:50:50
200 : Tue 19:51:25
300 : Tue 19:51:38
400 : Tue 19:51:47
500 : Tue 19:52:28
600 : Tue 19:52:54
700 : Tue 19:53:07
800 : Tue 19:53:56
900 : Tue 19:54:28
1000 : Tue 19:54:51
1100 : Tue 19:55:22
1200 : Tue 19:56:01
1300 : Tue 19:56:27
1400 : Tue 19:56:55
1500 : Tue 19:57:40
1600 : Tue 19:58:21
1700 : Tue 19:58:41
1800 : Tue 19:59:36
1900 : Tue 20:00:05
2000 : Tue 20:00:24
2100 : Tue 20:01:22
2200 : Tue 20:01:46
2300 : Tue 20:02:19
2400 : Tue 20:02:43
2500 : Tue 20:02:43
2600 : Tue 20:02:46
2700 : Tue 20:03:24
2800 : Tue 20:04:15
2900 : Tue 20:04:37
3000 : Tue 20:05:20
3100 : Tue 20:05:31
3200 : Tue 20:06:07
3300 : Tue 20:06:35
3400 : Tue 20:06:36
3500 : Tue 20:07:15
3600 : Tue 20:08:02
3700 : Tue 20:08:16
3800 : Tue 20:08:46
3900 : Tue 20:09:11
4000 : Tue 20:09:38
4100 : Tue 20:09:42
4200 : Tue 20:09:42
4300 : Tue 20:10:08
4400 : Tue 20:10:46
4500 : Tue 20:11:40
4600 : Tue 20:11:47
4700 : Tue 20:12:25
4800 : Tue 20:12:48
4900 : Tue 20:12:48
5000 : Tue 20:13

In [18]:
# parsing HTML : get KEGG ID

pub_kegg = []
for i in rs:
    if type(i) != requests.models.Response:
        pub_kegg.append('null')
        continue
    elif i.text == u'[]':
        pub_kegg.append('null')
        continue
    else:
        pub_kegg.append(i.text.encode('ascii', 'ignore')[21:27])

In [20]:
# Update KEGG ID 
df['KEGGID'] = pub_kegg

In [21]:
df.to_csv('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/1_herb_pubchem_kegg.csv', index= False)

In [24]:
#Using STITTCH ID, Get KEGG ID

# 1. remove '\r\n' from stitch ID. Update new stitch_id
pre_stitch_id = df['stitchID'].tolist()
stitch_id = []
for i in pre_stitch_id:
    stitch_id.append(i.rstrip('\r\n'))
df['stitchID'] = stitch_id

In [27]:
# Get stitch_id which is not converted to KEGG ID
stitch = df[(df.KEGGID == 'null') & (df.stitchID != 'null')]
stitch_id = stitch['stitchID'].tolist()

In [30]:
# read the pair of STITCH ID - KEGG ID
df_stitch= pd.read_csv('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_KEGG.csv', index_col = False)

In [32]:
# find the KEGG ID of top-herb's STITCH id from STITCH ID - KEGG ID pair
sti_pair = []
for i in stitch_id:
    flag = df_stitch[df_stitch.stereo_chemical == i].empty
    if not flag:
        temp_df = df_stitch[df_stitch.stereo_chemical == i]
        #sti_pair.append(temp_df['stereo_chemical'].tolist() + temp_df['source_id'].tolist())
        sti_pair.append([temp_df['stereo_chemical'].tolist(), temp_df['source_id'].tolist()])
        

In [34]:
# Update KEGG ID from STITCH ID
for i in sti_pair:
    sti = i[0]
    kegg = i[1]
    temp_df2 = df[df.stitchID == sti[0]]
    idx = temp_df2.index.values.tolist()
    for j in idx:
        df.loc[j, 'KEGGID'] = kegg[0]

In [36]:
df.to_csv('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/2_herb_stitch_kegg.csv', index= False)

In [39]:
# get chemblID whichi is not converted to KEGG ID using PubChem, STITCH.
chembl = df[(df.KEGGID == 'null') & (df.chemblID != 'null')]
chembl_id = chembl['chemblID'].tolist()

In [42]:
# using UniChem REST API, convert chembl ID to KEGG ID
# 1. requests HTML

url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'

# list of r object(request result)
rs=[]
cnt = 0
print time.strftime('%a %H:%M:%S')
for i in chembl_id:
    if i != 'null':
        url = url_base + i + '/1/6'
        #scrap the HTML at the url
        try : 
            r = requests.get(url)
            rs.append(r)
            r.connection.close()
        except requests.ConnectionError:
            print url
            break
    else:
        rs.append('null')
    cnt = cnt + 1
    if cnt % 100 == 0 :
        print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

Wed 13:28:00
Wed 13:28:45


In [43]:
# parsing HTML : get KEGG ID
chembl_kegg = []
for i in rs:
    if type(i) != requests.models.Response:
        chembl_kegg.append('null')
        continue
    elif i.text == u'[]':
        chembl_kegg.append('null')
        continue
    else:
        chembl_kegg.append(i.text.encode('ascii', 'ignore')[21:27])

In [47]:
chembl_idx = chembl.index.tolist()

In [63]:
# Update KEGG ID from chemblID
for i, j in zip(chembl_kegg, chembl_idx):
    df.loc[j, 'KEGGID'] = i

In [67]:
df.to_csv('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/3_herb_chembl_kegg.csv', index= False)

In [69]:
cas = df[(df.KEGGID == 'null') & (df.casID != 'null')]

In [71]:
cas_id = cas['casID'].tolist()

In [145]:
cas_idx = cas.index.tolist()
#len(cas_idx)

2417

In [73]:
cas_id

['182565-81-7',
 '220210-98-0',
 '174416-97-8',
 '76689-95-7',
 '33880-83-0',
 '67023-79-4',
 '67023-80-7',
 '86160-86-3',
 '108-39-4',
 '106-44-5',
 '1960-12-08',
 '100-51-6',
 '140-11-4',
 '28808-62-0',
 '16838-01-0',
 '2753-30-2',
 '220137-96-2',
 '21688-61-9',
 '469-39-6',
 '4657-58-3',
 '57589-60-3',
 '57589-59-0',
 '76689-93-5',
 '136880-81-4',
 '7392-19-0',
 '128533-21-1',
 '64118-73-6',
 '89915-37-7',
 '176181-83-2',
 '185414-86-2',
 '185414-90-8',
 '80787-59-3',
 '33512-38-8',
 '76-78-8',
 '90375-52-3',
 '82470-74-4',
 '80648-28-8',
 '80180-30-9',
 '76-77-7',
 '4616-50-6',
 '50886-57-2',
 '1448-22-2',
 '479-43-6',
 '66762-19-4',
 '29913-86-8',
 '509-24-0',
 '30333-81-4',
 '110-54-3',
 '1072-83-9',
 '23180-57-6',
 '71-36-3',
 '122-78-1',
 '121-29-9',
 '39011-90-0',
 '121-21-1',
 '470-82-6',
 '89-83-8',
 '139954-00-0',
 '98751-79-2',
 '80454-42-8',
 '1957-11-04',
 '124-18-5',
 '506-32-1',
 '55290-63-6',
 '12/1/3691',
 '201012-14-8',
 '193757-67-4',
 '470-55-3',
 '125196-77-2',
 

In [125]:
rs[len(rs)-3]

[{u'fromIdentifier': u'CAS',
  u'result': [],
  u'searchTerm': u'-',
  u'toIdentifier': u'KEGG'}]

In [122]:
print i
print len(rs)
print url
print r

-
1046
http://cts.fiehnlab.ucdavis.edu/service/convert/CAS/KEGG/-
<addinfourl at 139768166072688 whose fp = None>


In [126]:
# using chemical translation service REST webservices, convert casID to KEGG ID
# 1. requests HTML

url_base = 'http://cts.fiehnlab.ucdavis.edu/service/convert/CAS/KEGG/'

# list of r object(request result)
rs=[]
cnt = 0
print time.strftime('%a %H:%M:%S')
for i in cas_id:
    try : 
        url = url_base + i
        #scrap the HTML at the url
        r = urllib2.urlopen(url)
        result = json.load(r)
        rs.append(result)
        r.close()
    except urllib2.HTTPError:
        rs.append('null')
    except urllib2.socket.error:
        print i
        print len(rs)
        print url
        print r
        print cas_id[len(rs)-1]
        print rs[len(rs)-1]
        break
    
    cnt = cnt + 1
    if cnt % 100 == 0 :
        print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

Wed 15:56:29
100 : Wed 15:58:04
200 : Wed 16:00:22
300 : Wed 16:02:22
400 : Wed 16:03:26
500 : Wed 16:05:02
600 : Wed 16:07:14
6985-35-9
642
http://cts.fiehnlab.ucdavis.edu/service/convert/CAS/KEGG/6985-35-9
<addinfourl at 139768166864944 whose fp = None>
25152-84-5
[{u'searchTerm': u'25152-84-5', u'toIdentifier': u'KEGG', u'result': [], u'fromIdentifier': u'CAS'}]
Wed 16:08:13


In [135]:
# using chemical translation service REST webservices, convert casID to KEGG ID
# 1. requests HTML

url_base = 'http://cts.fiehnlab.ucdavis.edu/service/convert/CAS/KEGG/'

# list of r object(request result)
#rs=[]
#cnt = 0
print time.strftime('%a %H:%M:%S')
for i in cas_id[len(rs):]:
    try : 
        url = url_base + i
        #scrap the HTML at the url
        r = urllib2.urlopen(url)
        result = json.load(r)
        rs.append(result)
        r.close()
    except urllib2.HTTPError:
        rs.append('null')
    except urllib2.socket.error:
        print i
        print len(rs)
        print url
        print r
        print cas_id[len(rs)-1]
        print rs[len(rs)-1]
        break
    
    cnt = cnt + 1
    if cnt % 100 == 0 :
        print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

Wed 17:41:43
2300 : Wed 17:46:29
2400 : Wed 17:48:41
Wed 17:49:29


In [141]:
# parsing HTML : get KEGG ID
cas_kegg = []
for i in rs:
    if i != 'null':
        if i[0]['result'] == []:
            cas_kegg.append('null')
            continue
        else:
            cas_kegg.append(i[0]['result'][0].encode('ascii'))
    else:
        cas_kegg.append('null')

In [146]:
# Update KEGG ID from chemblID
for i, j in zip(cas_kegg, cas_idx):
    df.loc[j, 'KEGGID'] = i

In [150]:
df.to_csv('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/4_herb_cas_kegg.tsv', sep='\t')

In [151]:
df_only = df[df.KEGGID != 'null']

In [152]:
df_only.to_csv('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/5_herb_kegg(final).tsv', sep='\t')

In [153]:
len(df)

37123

In [154]:
len(df_only)

3391